### Python wheel created from trase utility code

In [0]:
%pip install /dbfs/FileStore/jars/f90d9f8d_fd2a_4845_b5b0_cdfd9213b291/trase_tools-0.1.0-py3-none-any.whl

### import works

In [0]:
import trase_tools
from trase_tools.tools.utilities.helpers import *
i = Identifier("Cote D'Ivoire")
print(i.upper_space)

In [0]:
import dlt
import io
import numpy as np

from pyspark.sql.functions import *
import pyspark.pandas as ps
import pandas as pd
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true")
#spark.conf.set("spark.databricks.delta.schema.overwriteSchema.enabled","true")



## Functions that replace the load and write helper functions from original Trase code. 
__The aim is to generalize the helper functions with delta code, for seamless integration of Delta Live Tables__

In [0]:

def get_df_clean(key,sep=";",encoding="latin1",bucket_name="s3a://uutrase/data/trase-uppsala/",type=None,skiprows=None):
    
    return spark.createDataFrame(remove_whitespace(spark.read.option("delimiter", sep)
                             .option("encoding","UTF-8")
                             .option("header", "true")
                             .option("multiline","true")
                             .csv(bucket_name + key).toPandas()))
    
#get data from existing table

    
    
def remove_whitespace(df):
    for col in df.columns:
        if df[col].dtype in (np.object, np.str):
            df[col] = df[col].str.strip()
            
    return df

In [0]:

def get_df_while_creating_table(key,sep=";",encoding="latin1",bucket_name="s3a://uutrase/data/trase-uppsala/",type=None,skiprows=None):
        table_name = key.split("/")[-1].split(".")[0].replace("-","_")
        df = get_df_clean(key,sep,encoding,bucket_name,skiprows)
        spark_df = spark.createDataFrame(df)
        @dlt.create_table(name = table_name)
        def create_trase_table():
            return spark_df.select([col(c).alias(c.replace(" ", "_")) for c in spark_df.columns]) 
        return df
            
        
def get_df(table_name):
#    return ps.DataFrame(dlt.read("city_df").select("*")).to_pandas()
    return ps.DataFrame(dlt.read(table_name)).to_pandas()

def delta_to_pandas(table_name):
    py_pandas_df = pd.DataFrame(dlt.read(table_name))
    print("py_pandas_df",py_pandas_df)
    return py_pandas_df

def zero_round():
    @dlt.table(name="first_table")
    def df_table():
        pandas_df = delta_to_pandas("city_df")

        print("regular pandas: ",pandas_df)
        
        return spark.createDataFrame(pandas_df)



## Trase script, slightly modified

In [0]:
def shrimp_processing():
    city_df = get_df("city_df")
    parish_df = get_df("parish_df")
    print("parish_df",len(parish_df))
    city_df = pd.merge(
        city_df,
        parish_df,
        how="left",
        left_on=["PARISH", "CANTON"],
        right_on=["PARISH_NAME", "CANTON_NAME"],
    )
    print("merged city_df",len(city_df))
    city_df["PARISH_TRASE_ID"] = city_df["TRASE_ID"]
    city_df = city_df[["PARISH_TRASE_ID", "CITY", "REGION", "CANTON"]]


    eu_df = get_df("eu_df")
    print("eu_df",len(eu_df))
    eu_codes = eu_df.CODE.tolist()

    china_df = get_df("china_df")
    china_df["CODE"] = china_df["APPROVALNO_CLEAN"]
    print("china_df",len(china_df))
    china_codes = china_df.CODE.tolist()

    df1 = get_df("df1")
    df2 = get_df("df2")
    df3 = get_df("df3")

    df3 = df3[
        ["CODIGO", "NOMBRE", "DIRECCION_ESTABLECIMIENTO", "TELEFONOS", "CONTACTO"]
    ].rename(columns={"CODIGO": "CODIGO"})
    # Remove duplicates in joint dataset - dfpesca2017.nunique()
    print("df1",len(df1))
    print("df2",len(df2))
    print("df3",len(df3))
    dfpesca2017 = pd.concat([df1, df2])
    dfpesca2017.drop_duplicates(
        subset=["CODIGO", "NOMBRE", "DIRECCION_ESTABLECIMIENTO", "CONTACTO"], inplace=True
    )
    # dfpesca2017 = dfpesca2017[[]]
    df2017 = pd.concat([dfpesca2017, df3])
    print(len(df2017["CODIGO"].unique()))
    print(len(df2017))

    print(len(df2017["CODIGO"].unique()))
    print(len(df2017))


    assert len(df2017["CODIGO"].unique()) == len(df2017)
    for column in ["NOMBRE", "DIRECCION_ESTABLECIMIENTO", "TELEFONOS", "CONTACTO"]:
        df2017[column] = df2017[column].str.replace(r"\r", " ")
        df2017[column] = df2017[column].str.replace(r"\n", " ")

    df1_2018 = get_df("df1_2018")
    df2_2018 = get_df("df2_2018")
    df2018 = pd.concat([df1_2018, df2_2018])
    assert len(df2018["CODIGO"].unique()) == len(df2018)
    for column in ["NOMBRE", "DIRECCION_ESTABLECIMIENTO", "TELEFONOS", "CONTACTO"]:
        #print(df2018.columns)
        df2018[column] = df2018[column].str.replace(r"\r", " ")
        df2018[column] = df2018[column].str.replace(r"\n", " ")

    dfs = []
    for year, df in {2017: df2017, 2018: df2018}.items():
        df.rename(
            columns={
                "CODIGO": "CODE",
                "NOMBRE": "NAME",
                "DIRECCION_ESTABLECIMIENTO": "ADDRESS",
                "DIRECCION_ESTABLECIMIENTO": "ADDRESS",
                "TELEFONOS": "TELEPHONE",
                "CONTACTO": "CONTACT",
            },
            inplace=True,
        )

        # Compare with EU facilities
        all_codes = df.CODE.tolist()
        for eu_code in eu_codes:
            if eu_code not in all_codes:
                if f"PPA-{eu_code}" not in all_codes:
                    if f"PA-{eu_code}" not in all_codes:
                        print(eu_code)
                        # TODO: Half of them are 2019, we need to remove these facilities
                        print(
                            f"eu facility: {eu_code}, {eu_df[eu_df.CODE == eu_code].YEAR} "
                        )

        # Match with EU dataset by code
        df["SIMPLE_CODE"] = df.CODE.apply(lambda value: value.split("-")[-1]).astype(int)
        eu_df["SIMPLE_CODE"] = eu_df.CODE.apply(lambda value: value.split("-")[-1]).astype(
            int
        )
        tdf = pd.merge(df, eu_df, how="left", left_on="SIMPLE_CODE", right_on="SIMPLE_CODE")

        # If there is a match in eu dataset flag this in a new column
        tdf["EU_CERTIFIED"] = np.where(tdf.CODE_y.isna(), "FALSE", "TRUE")
        tdf["TYPE"] = np.where(tdf.CODE_x.str.startswith("PPA-"), "F Aq", "Aq")
        tdf.rename(
            columns={
                "CODE_x": "CODE",
                "NAME_x": "NAME",
                "NAME_y": "EU_NAME_SYNONYM",
                "COMMENT": "EU_TYPE",
            },
            inplace=True,
        )
        tdf = tdf[
            [
                "CODE",
                "NAME",
                "ADDRESS",
                "TELEPHONE",
                "CONTACT",
                "SIMPLE_CODE",
                "CITY",
                "REGION",
                "EU_NAME_SYNONYM",
                "EU_TYPE",
                "EU_CERTIFIED",
                "YEAR",
                "TYPE",
            ]
        ]
        for china_code in china_codes:
            if china_code not in all_codes:
                if f"PPA-{china_code}" not in all_codes:
                    if f"PA-{china_code}" not in all_codes:
                        print(f"chinese facility: {china_code}")

        # Match with EU dataset by code
        #tdf["SIMPLE_CODE"] = tdf["SIMPLE_CODE"].astype("int64")
        china_df["CODE"] = china_df["CODE"].astype(int)
        tdf = pd.merge(tdf, china_df, how="left", left_on="SIMPLE_CODE", right_on="CODE")
        # If there is a match in china dataset flag this in a new column
        tdf["CHINA_CERTIFIED"] = np.where(tdf.CODE_y.isna(), "FALSE", "TRUE")
        # Update
        tdf["CITY"] = np.where(tdf.CITY.isna(), tdf["City_County"].str.upper(), tdf.CITY)
        type_dict = {
            "A": "Aq",
            "A - FISHERY PRODUCTS": "F Aq",
            "FISHERY PRODUCTS": "F",
            "AQUACULTURE PRODUCTS/FISHERY PRODUCTS": "F Aq",
            "AQUACULTURE PRODUCTS": "Aq",
            "A - AQUACULTURE PRODUCTS": "Aq",
        }
        tdf.Remark = tdf.Remark.map(type_dict)
        tdf["TYPE"] = np.where(tdf.TYPE.isna(), tdf.Remark, tdf.TYPE)
        tdf.rename(
            columns={"CODE_x": "CODE", "NAME_x": "NAME", "NAME_y": "CHINA_NAME_SYNONYM"},
            inplace=True,
        )
        tdf = tdf[
            [
                "CODE",
                "NAME",
                "ADDRESS",
                "TELEPHONE",
                "CONTACT",
                "SIMPLE_CODE",
                "CITY",
                "REGION",
                "EU_NAME_SYNONYM",
                "EU_TYPE",
                "EU_CERTIFIED",
                "CHINA_NAME_SYNONYM",
                "CHINA_CERTIFIED",
                "YEAR",
                "TYPE",
            ]
        ]

        # TODO extract names that do not match : SYNONYMS
        # names = tdf[tdf.NAME_x != tdf.NAME_y]
        # Look at non_matches and merge in cantons from city dictionary
        tdf = pd.merge(tdf, city_df, how="left", left_on="CITY", right_on="CITY")

        # Rename columns and rejigg
        tdf.rename(
            columns={
                "CODE_x": "CODE",
                "NAME_x": "NAME",
                "NAME_y": "CHINA_NAME_SYNONYM",
                "REGION_x": "REGION",
            },
            inplace=True,
        )
        tdf = tdf[
            [
                "CODE",
                "NAME",
                "ADDRESS",
                "TELEPHONE",
                "CONTACT",
                "SIMPLE_CODE",
                "CITY",
                "PARISH_TRASE_ID",
                "REGION",
                "CANTON",
                "EU_NAME_SYNONYM",
                "EU_TYPE",
                "EU_CERTIFIED",
                "CHINA_NAME_SYNONYM",
                "CHINA_CERTIFIED",
                "TYPE",
            ]
        ]

        # Final push
        print(
            f"There are {len(tdf[tdf.CANTON.isna()])} addresses to check out of {len(tdf)}."
        )
        dfs.append(tdf)

    # Combine the two files
    concat_df = pd.concat(dfs)
    print("concat_df: ",len(concat_df))
    final_df = concat_df.drop_duplicates(
        subset=[
            "CODE",
            "NAME",
            "SIMPLE_CODE",
            "CITY",
            "PARISH_TRASE_ID",
            "REGION",
            "CANTON",
            "EU_NAME_SYNONYM",
            "EU_TYPE",
            "EU_CERTIFIED",
            "CHINA_CERTIFIED",
            "TYPE",
        ]
    ).reset_index(drop=True)
    #print(len(final_df))
    #print(len(final_df.CODE.unique()))



    assert len(final_df) == len(final_df.CODE.unique())

    # Add source column
    final_df["SOURCE"] = "www.acuaculturaypesca.gob.ec"
    
    print("final_df: ",final_df)
    return ps.DataFrame(final_df)

def create_and_read_table():
    @dlt.table(name="best_table")
    def read_table():
        a = dlt.read("city_df")
        return a

## Creation of Delta Live Tables

In [0]:
@dlt.table(name="city_df")
def get_city_df():
    return get_df_clean("ecuador/spatial/BOUNDARIES_temp/city_to_parish_2.csv", sep=",", encoding="utf8")

@dlt.table(name="china_df")
def get_china_df():
    return get_df_clean("ecuador/logistics/shrimp_processing_facilities/out/china/CHINA_matches.csv",sep=",",encoding="utf8")

@dlt.table(name="parish_df")
def get_parish_df():
    return get_df_clean("ecuador/spatial/BOUNDARIES_out/ec_parishes.csv", sep=";", encoding="utf8")


@dlt.table(name="eu_df")
def get_eu_df():
    return get_df_clean("ecuador/logistics/shrimp_processing_facilities/out/eu/out/ec_processing_facilities_eu.csv",encoding="utf8")

@dlt.table(name="df1")
def get_df1():
    return get_df_clean("ecuador/logistics/shrimp_processing_facilities/acuaculturaypesca/3-cleaned/2_ACTUALIZA_PROCESA_PESCA_ACUA_22DICIEMBRE2017.csv", encoding="utf8")

@dlt.table(name="df2")
def get_df2():
    return get_df_clean("ecuador/logistics/shrimp_processing_facilities/acuaculturaypesca/3-cleaned/2_tabula-ACTUALIZA_PROCESA_PESCA_ACUA_24OCTUBRE2017.csv", encoding="utf8")

@dlt.table(name="df3")
def get_df3():
    return get_df_clean("ecuador/logistics/shrimp_processing_facilities/acuaculturaypesca/3-cleaned/2_ACTUALIZA_PROCESA_ACUACULTURA_12OCTUBRE2017.csv", encoding="utf8")

@dlt.table(name="df1_2018")
def get_df1_2018():
    return get_df_clean("ecuador/logistics/shrimp_processing_facilities/acuaculturaypesca/3-cleaned/2_tabula-ACTUALIZA_PROCESA_PESCA_ACUA_13AGOSTO2018.csv", encoding="utf8")


@dlt.table(name="df2_2018")
def get_df2_2018():
    return get_df_clean("ecuador/logistics/shrimp_processing_facilities/acuaculturaypesca/3-cleaned/2_tabula-ACTUALIZA_PROCESA_ACUACULTURA_31AGOSTO2018.csv", encoding="utf8")

#zero_round()
@dlt.table(name="first_round")
def shrimp_man():
    return shrimp_processing()


    